<a href="https://colab.research.google.com/github/zakstucke/geo-sentiment-analysis/blob/master/FinalVaccineChoropleth.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import plotly.express as px
import datetime

# OWID vaccine data source
# vaccine_data = "https://raw.githubusercontent.com/owid/covid-19-data/master/public/data/vaccinations/vaccinations.csv"
# OWIDdf = pd.read_csv(vaccine_data)


# load in processed data
pd.options.mode.chained_assignment = None  # default='warn' this is needed for false positives warnings on reassigning pd dataframe

# Processed cleaned sentiment source
WHOfile = "https://media.githubusercontent.com/media/zakstucke/geo-sentiment-analysis/master/processed_data/final_csv.txt?token=ARLQAMUI3FRYOTEC4IF6OX3CMEHHE"
WHOdf = pd.read_csv(WHOfile)
WHOdf

# No sentiment, but has cases & vaccines
big_WHO_file = "https://raw.githubusercontent.com/zakstucke/geo-sentiment-analysis/master/data/WHO_covid_data_by_COUNTRY.csv?token=GHSAT0AAAAAABTWTNX6BY2NR723WHGYHFJSYTBCALQ"
cols = ["iso_code, date, total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,\
 people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred"]
big_WHO_df = pd.read_csv(big_WHO_file)

# iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,total_cases_per_million,new_cases_per_million,new_cases_smoothed_per_million,
# total_deaths_per_million,new_deaths_per_million,new_deaths_smoothed_per_million,reproduction_rate,icu_patients,icu_patients_per_million,hosp_patients,hosp_patients_per_million,weekly_icu_admissions,
# weekly_icu_admissions_per_million,weekly_hosp_admissions,weekly_hosp_admissions_per_million,total_tests,new_tests,total_tests_per_thousand,new_tests_per_thousand,new_tests_smoothed,new_tests_smoothed_per_thousand,
# positive_rate,tests_per_case,tests_units,total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,
# people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,new_people_vaccinated_smoothed_per_hundred,stringency_index,population,population_density,
# median_age,aged_65_older,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy,
# human_development_index,excess_mortality_cumulative_absolute,excess_mortality_cumulative,excess_mortality,excess_mortality_cumulative_per_million

In [137]:
# Columns to look at:
# total_vaccinations,people_vaccinated,people_fully_vaccinated,total_boosters,new_vaccinations,new_vaccinations_smoothed,total_vaccinations_per_hundred,people_vaccinated_per_hundred,\
# people_fully_vaccinated_per_hundred,total_boosters_per_hundred,new_vaccinations_smoothed_per_million,new_people_vaccinated_smoothed,"


col = "people_vaccinated_per_hundred" # Column definitely produces a working output

In [138]:
df = big_WHO_df[big_WHO_df["iso_code"] != "GIB"] # drop GIB as outlier
# df[["iso_code","people_fully_vaccinated_per_hundred"]].dropna().sort_values(by = "people_fully_vaccinated_per_hundred")
df2 = df[["iso_code","date", col]] # Take columns I want
df2 = df2.groupby(["date","iso_code"]).agg("max") # Take the maximum of a given date for duplicated dates
df2 = df2.reset_index(level=0).reset_index(level=0) # Get column back as a column after aggregation
df2 = df2.sort_values(["iso_code","date"]) # sort so that we can forward fill
df2 = df2.loc[df2["date"] > "2020-2-24"]  # remove old data so that the x axis scales nicely

# Forward fill using a for loop
# In an ideal scenario, a built in method would be used. If you have one feel free to simplify this
df2["iso_code_lag"] = df2["iso_code"].shift(1)
df2 = df2.reset_index() 
x = 0
for index, row in df2.iterrows():
  if (row["iso_code"] != row["iso_code_lag"]):
    x = 0
  if (pd.isna(row[col])):
    df2.at[index,col]=x
  else:
    x = row[col]
df2


#
# Old Merge method [ignore/needs fix]
#

# over_time = features.sort_values("date")
# # Unique
# dates = over_time["date"].unique()
# print(f"unique dates = {len(dates)}")
# countries = over_time["location"].unique()
# print(f"unique countries = {len(countries)}")
# test = over_time
# a = pd.concat([pd.DataFrame([[j,i]], columns=['locations','date']) for i in dates for j in countries])                

# b = pd.merge(a, features,  how='left', left_on=['iso_code','date'], right_on = ['location','date'])
# b = b.sort_values("date")
# b = b.fillna(method = 'ffill') #.replace(to_replace=0, method='ffill')
# b["people_fully_vaccinated_per_hundred"]


# extra_dates = pd.concat([pd.DataFrame([i], columns=['date']) for i in dates], ignore_index=True)
# extra_dates.shape
# over_time = features.sort_values("date")
# # features.columns
# over_time[col].replace(to_replace=0, method='ffill')
# over_time

,index,iso_code,date,people_vaccinated_per_hundred,iso_code_lag
0,66475,ABW,2021-01-01,0.00,NaN
1,66697,ABW,2021-01-02,0.00,ABW
2,66919,ABW,2021-01-03,0.00,ABW
3,67141,ABW,2021-01-04,0.00,ABW
4,67363,ABW,2021-01-05,0.00,ABW
...,...,...,...,...,...
102712,168283,ZWE,2022-03-16,29.22,ZWE
102713,168510,ZWE,2022-03-17,29.24,ZWE
102714,168737,ZWE,2022-03-18,29.27,ZWE
102715,168964,ZWE,2022-03-19,29.27,ZWE


In [ ]:
#Take specific column to check values
df2.loc[df2["iso_code"] =="ARG"].values.tolist()

In [141]:
def create_choropleth(df2, col):
  max_total = df2[col].max()
  min_total = df2[col].min()

  # Chloropleth
  fig = px.choropleth(df2, # dataset to use
                    locations="iso_code", # column which includes 3 letter country code
                    locationmode = "ISO-3",
                    animation_frame = "date",
                    color=col, # column which dictates the colour of the map
                   # hover_name="location", # column to add to hover information
                    range_color=(min_total, max_total), #,# range of the colour scale
                    color_continuous_scale=["red","yellow","green","blue"]
  )
  fig.show()

In [142]:
create_choropleth(df2, col)

In [148]:
# New Vaccine updtake
col = "new_vaccinations"

df = big_WHO_df[big_WHO_df["iso_code"] != "GIB"] # drop GIB as outlier

df2 = df[["iso_code","date", col]] # Take columns I want
df2 = df2.loc[df2["date"] > "2020-2-24"]  # remove old data so that the x axis scales nicely
df2["log_col"] = np.log(df2[col])

df2[col].max()



max_total = df2["log_col"].max()
min_total = df2["log_col"].min()

# Chloropleth
fig = px.choropleth(df2, # dataset to use
                    locations="iso_code", # column which includes 3 letter country code
                    locationmode = "ISO-3",
                    animation_frame = "date",
                    color="log_col", # column which dictates the colour of the map
                   # hover_name="location", # column to add to hover information
                    range_color=(min_total, max_total), #,# range of the colour scale
                    color_continuous_scale=["red","yellow","green","blue"]
)
fig.show()



# df2 = df2.groupby(["date","iso_code"]).agg("max") # Take the maximum of a given date for duplicated dates
# df2 = df2.reset_index(level=0).reset_index(level=0) # Get column back as a column after aggregation
# df2 = df2.sort_values(["iso_code","date"]) # sort so that we can forward fill
# df2 = df2.loc[df2["date"] > "2020-2-24"]  # remove old data so that the x axis scales nicely

# # Forward fill using a for loop
# # In an ideal scenario, a built in method would be used. If you have one feel free to simplify this
# df2["iso_code_lag"] = df2["iso_code"].shift(1)
# df2 = df2.reset_index() 
# x = 0
# for index, row in df2.iterrows():
#   if (row["iso_code"] != row["iso_code_lag"]):
#     x = 0
#   if (pd.isna(row[col])):
#     df2.at[index,col]=x
#   else:
#     x = row[col]
# df2

/usr/local/lib/python3.7/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning:

divide by zero encountered in log

